In [ ]:
# Project: AgentsVille Trip Planner
#
# Welcome to your final project! In this notebook, you'll build the "AgentsVille Trip Planner,"
# an AI-powered assistant that helps users plan trips to the imaginary city of AgentsVille.
# You will apply the prompting techniques and agentic reasoning concepts you've learned
# throughout the course.

# Project Goal:
# 1. Generate an initial, detailed travel itinerary based on user preferences.
# 2. Enhance this itinerary using an AI agent that can use (simulated) tools.

# --- 0. Setup ---
# Import necessary libraries and set up the OpenAI client and helper functions.

# Import necessary libraries
import os
import json
import re
from enum import Enum
from typing import List, Dict, Any, Optional, Literal

from openai import OpenAI
# Ensure you have pydantic installed: pip install pydantic
from pydantic import BaseModel, Field, ValidationError
from IPython.display import Markdown, display

# Attempt to import from project_lib, provide stubs if not found
try:
    from project_lib import (
        TravelPreferences, TravelItinerary, Activity, DayPlan, PackingListItem, # Pydantic Models
        available_tools, # Dictionary of tool schemas and functions
        # Tool implementations are called via available_tools dictionary
        LLMToolCall # Pydantic model for tool calls
    )
    print("Successfully imported from project_lib.py")
except ImportError:
    print("Warning: project_lib.py not found or some components are missing. Using placeholder Pydantic models and functions.\n",
          "Please ensure project_lib.py is in the same directory as this notebook for full functionality.")

    # Define dummy Pydantic models if project_lib is not available
    class TravelPreferences(BaseModel):
        destination: str
        duration_days: int
        travel_style: List[str]
        interests: List[str]
        budget: Literal["budget", "mid-range", "luxury"]
        specific_requests: Optional[str] = None

    class Activity(BaseModel):
        time: str
        description: str
        estimated_cost: Optional[str] = None
        details: Optional[str] = None

    class DayPlan(BaseModel):
        day_number: int
        theme: Optional[str] = None
        activities: List[Activity]

    class PackingListItem(BaseModel):
        item: str
        quantity: Optional[int] = 1
        notes: Optional[str] = None

    class TravelItinerary(BaseModel):
        trip_name: str
        destination: str
        duration_days: int
        daily_plans: List[DayPlan]
        suggested_packing_list: Optional[List[PackingListItem]] = None
        overall_estimated_budget: Optional[str] = None
        notes: Optional[str] = None

    class LLMToolCall(BaseModel): # For validating the LLM's desired tool call
        name: str
        arguments: Dict[str, Any]

    # Dummy tool implementations (these would be in project_lib.py)
    def get_weather_forecast(location: str, date: str) -> Dict[str, Any]:
        print(f"[Tool Stub] Called get_weather_forecast for {location} on {date}")
        return {"forecast": "Sunny", "temperature": "22C", "location": location, "date": date}

    def search_activities(location: str, interests: List[str], date: Optional[str] = None) -> List[Dict[str, Any]]:
        print(f"[Tool Stub] Called search_activities for {location} with interests {interests} on {date}")
        return [{"name": "Museum of Agentic Design", "description": "Explore the history of AI agents."}]

    def find_restaurants(location: str, cuisine_type: Optional[str] = None, price_range: Optional[str] = None) -> List[Dict[str, Any]]:
        print(f"[Tool Stub] Called find_restaurants for {location}, cuisine {cuisine_type}, price {price_range}")
        return [{"name": "The Prompt Cafe", "cuisine": cuisine_type if cuisine_type else "various"}]

    def book_hotel(location: str, check_in_date: str, check_out_date: str, preferences: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        print(f"[Tool Stub] Called book_hotel for {location} from {check_in_date} to {check_out_date}")
        return {"booking_confirmation": "HOTEL-STUB-CONFIRMED", "hotel_name": "The Grand Agent Hotel"}

    available_tools = {
        "get_weather_forecast": {
            "function": get_weather_forecast,
            "description": "Get the weather forecast for a specific location and date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state, e.g., San Francisco, CA"},
                    "date": {"type": "string", "description": "The date for the forecast, in YYYY-MM-DD format."}
                },
                "required": ["location", "date"]
            }
        },
        "search_activities": {
            "function": search_activities,
            "description": "Searches for activities based on location, interests, and optionally a date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The location to search for activities."},
                    "interests": {"type": "array", "items": {"type": "string"}, "description": "A list of interests to guide the activity search."},
                    "date": {"type": "string", "description": "Optional. The specific date for activities, in YYYY-MM-DD format."}
                },
                "required": ["location", "interests"]
            }
        },
        "find_restaurants": {
            "function": find_restaurants,
            "description": "Finds restaurants based on location, cuisine type, and price range.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The location to search for restaurants."},
                    "cuisine_type": {"type": "string", "description": "Optional. The desired type of cuisine (e.g., Italian, Mexican)."},
                    "price_range": {"type": "string", "description": "Optional. The desired price range (e.g., $, $$, $$$)."}
                },
                "required": ["location"]
            }
        },
        "book_hotel": {
            "function": book_hotel,
            "description": "Books a hotel based on location, dates, and preferences.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The location for the hotel."},
                    "check_in_date": {"type": "string", "description": "The check-in date in YYYY-MM-DD format."},
                    "check_out_date": {"type": "string", "description": "The check-out date in YYYY-MM-DD format."},
                    "preferences": {
                        "type": "object",
                        "description": "Optional. Dictionary of preferences like room type, amenities.",
                        "properties": {
                             "room_type": {"type": "string", "description": "e.g., King, Queen, Suite"},
                             "amenities": {"type": "array", "items": {"type": "string"}, "description": "e.g., ['pool', 'gym']"}
                        }
                    }
                },
                "required": ["location", "check_in_date", "check_out_date"]
            }
        }
    }

# OpenAI Client Setup
# TODO: Configure your OpenAI API Key
# Replace "**********" with your actual Vocareum OpenAI API Key.
# Alternatively, if your classroom workspace sets the OPENAI_API_KEY environment variable,
# you can use `api_key=os.getenv("OPENAI_API_KEY")`.

try:
    client = OpenAI(
        base_url="https://openai.vocareum.com/v1",
        api_key="**********"  # TODO: Replace "**********" with your key
    )
    print("OpenAI client configured (key needs to be set).")
except Exception as e:
    print(f"Error configuring OpenAI client: {e}")

# Define helper functions
class OpenAIModels(str, Enum):
    GPT_41_MINI = "gpt-4.1-mini"

MODEL = OpenAIModels.GPT_41_MINI

def get_completion(messages: List[Dict[str, Any]], model: str = MODEL, temperature: float = 0.2, response_format: Optional[Dict[str, str]] = None) -> Optional[Any]:
    """
    Function to get a completion from the OpenAI API.
    Returns the message object if tool_calls are present, otherwise the content string.
    """
    try:
        completion_params = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
        }
        if response_format: # For requesting JSON mode
            completion_params["response_format"] = response_format

        # For ReAct agent, we might need to pass tool schemas if using OpenAI's native tool calling.
        # For this project, we are guiding the LLM to output JSON for tool calls in the ACT step.
        # If using native tool calling, you would add 'tools' and 'tool_choice' parameters here.

        response = client.chat.completions.create(**completion_params)
        message = response.choices[0].message

        if message.tool_calls: # Check for native tool calls (OpenAI v1.x+)
            return message # Return the whole message object
        return message.content # Return just the text content
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None

def print_messages(messages: List[Dict[str, Any]]):
    """Helper function to print messages, including tool calls and tool results."""
    for message in messages:
        role = message.get('role', 'unknown').capitalize()
        content = message.get('content')

        display(Markdown(f"**{role}:**"))
        if content and isinstance(content, str):
            display(Markdown(content))

        # Handling new tool_calls format (list of objects)
        if message.get('role') == 'assistant' and message.get('tool_calls'):
            display(Markdown("  *Wants to call tools:*"))
            for tc in message['tool_calls']:
                function_info = tc.get('function', {})
                display(Markdown(f"  - **Tool Call ID:** {tc.get('id')}"))
                display(Markdown(f"    - **Function:** `{function_info.get('name')}`"))
                display(Markdown(f"    - **Arguments:** `{function_info.get('arguments')}`"))

        if message.get('role') == 'tool':
            display(Markdown(f"  *Tool Call Result (ID: {message.get('tool_call_id')}) for `{message.get('name')}`:*"))
            if content:
                 display(Markdown(f"    `{str(content)}`"))
        print("---")

# --- 1. Defining User Preferences (Input) ---
user_preferences_data = {
    "destination": "AgentsVille",
    "duration_days": 3,
    "travel_style": ["culture", "history", "local experiences"],
    "interests": ["museums", "historical landmarks", "local markets", "street food"],
    "budget": "mid-range",
    "specific_requests": "I'd like to visit at least one major museum and try authentic local cuisine. I prefer using public transport or walking."
}

try:
    user_prefs = TravelPreferences(**user_preferences_data)
    print("User Preferences (Validated):")
    print(user_prefs.model_dump_json(indent=2))
except ValidationError as e:
    print(f"Error validating user preferences: {e}")
    user_prefs = None

# --- 2. Initial Itinerary Generation (CoT & Structured Output) ---
# Reference: TravelItinerary Pydantic Model (defined in project_lib.py or stub above)

# TODO: Craft the system prompt for initial itinerary generation.
# This prompt must instruct the LLM to act as a detailed travel planner for AgentsVille.
# It needs to consider all aspects of the `user_prefs`.
# The output MUST be a JSON object that validates against the `TravelItinerary` Pydantic model.
# Guide the LLM to create a coherent day-by-day plan, including activities,
# a suggested packing list, an overall estimated budget, and relevant notes.
# Remind it to ONLY output the JSON object.
# Apply principles of Chain-of-Thought by asking for a comprehensive, well-reasoned plan
# that covers all requirements.

system_prompt_generate_itinerary = """
********** TODO: DEFINE YOUR SYSTEM PROMPT HERE **********
Remember to specify the role, task, detailed output JSON structure (matching TravelItinerary),
and instructions to consider all user preferences and output ONLY JSON.
The JSON structure to follow is:
{
  "trip_name": "string", "destination": "string", "duration_days": "integer",
  "daily_plans": [ { "day_number": "integer", "theme": "string (optional)",
                   "activities": [ { "time": "string", "description": "string",
                                     "estimated_cost": "string (optional)",
                                     "details": "string (optional)" } ] } ],
  "suggested_packing_list": [ { "item": "string", "quantity": "integer (optional)",
                               "notes": "string (optional)" } ],
  "overall_estimated_budget": "string (optional)",
  "notes": "string (optional)"
}
"""

user_prompt_itinerary_request = f"""
Please generate a travel itinerary based on the following preferences:
{user_prefs.model_dump_json(indent=2) if user_prefs else "User preferences not available."}
"""

print("\\nGenerating initial itinerary...")
initial_itinerary: Optional[TravelItinerary] = None
raw_llm_itinerary_json_str = None

if user_prefs:
    messages_itinerary = [
        {"role": "system", "content": system_prompt_generate_itinerary},
        {"role": "user", "content": user_prompt_itinerary_request}
    ]
    # We expect a JSON object directly due to response_format
    llm_output_for_itinerary = get_completion(messages_itinerary, response_format={"type": "json_object"})

    if isinstance(llm_output_for_itinerary, str):
        raw_llm_itinerary_json_str = llm_output_for_itinerary
        try:
            initial_itinerary_json = json.loads(raw_llm_itinerary_json_str)
            initial_itinerary = TravelItinerary(**initial_itinerary_json)
            print("\\nInitial Itinerary Generated and Validated Successfully!")
            display(Markdown(f"### Trip Name: {initial_itinerary.trip_name}"))
            # print(initial_itinerary.model_dump_json(indent=2)) # For full details
        except (ValidationError, json.JSONDecodeError) as e:
            print(f"\\nError processing itinerary JSON: {e}")
            print(f"\\nLLM Response was:\\n{raw_llm_itinerary_json_str}")
    else:
        print("\\nFailed to generate initial itinerary (unexpected response type or None).")
else:
    print("User preferences not loaded, cannot generate itinerary.")


# --- 3. Tool-Using Agent for Enhancements (ReAct) ---
# Now, create an AI agent that uses tools to answer follow-up questions or enhance the itinerary.

print("\\nSchema for 'search_activities' tool (from available_tools):")
if "search_activities" in available_tools: # Check if stub or real lib loaded it
    print(json.dumps(available_tools['search_activities']['parameters'], indent=2))

# TODO: Craft the system prompt for the ReAct agent.
# This is a complex prompt and a core part of the project.
# Instructions for system_prompt_react_agent:
# 1. Role: 'AI Travel Assistant for AgentsVille'.
# 2. Context: It will have an existing itinerary (from user message) and its goal is to answer questions or enhance it using tools.
# 3. THINK-ACT-OBSERVE Cycle:
#    - THINK: Briefly outline its plan and reasoning for choosing an action/tool.
#    - ACT:
#        - If using tool(s), output *only* a valid JSON list of tool call objects: `[{"name": "tool_name", "arguments": {"arg1": "value1"}}]`
#        - If providing a final answer, output `FINAL ANSWER:` followed by the text.
# 4. Tool List: Provide the LLM with the list of available tools and their schemas (use `available_tools` to construct this part of the prompt).
# 5. Examples: Provide clear examples of a THINK-ACT (tool call) sequence AND a THINK-FINAL ANSWER sequence.

system_prompt_react_agent = f\"\"\"
********** TODO: DEFINE YOUR REACT AGENT SYSTEM PROMPT HERE **********

Remember to include:
- The agent's role.
- Instructions on the THINK-ACT-OBSERVE cycle.
- The exact format for ACT tool calls (JSON list of LLMToolCall-like objects).
- The exact format for FINAL ANSWER.
- The full list of AVAILABLE TOOLS with their names, descriptions, and parameter schemas
  (formatted from the `available_tools` dictionary).
- At least one clear example of a tool-calling interaction (User -> AI THINK/ACT -> User OBSERVATION -> AI THINK/FINAL ANSWER).
- At least one clear example of a direct final answer (User -> AI THINK/FINAL ANSWER).
\"\"\"

# --- Simulate ReAct Agent Interaction ---
react_messages: List[Dict[str, Any]] = [
    {"role": "system", "content": system_prompt_react_agent}
]

if initial_itinerary:
    current_itinerary_str = initial_itinerary.model_dump_json(indent=2)
    follow_up_request = "Can you find a unique local craft workshop for the afternoon of Day 2? Also, what will the weather be like on Day 1 of my trip, assuming Day 1 is 2025-10-26?"

    user_message_content_for_react = f\"\"\"Here is the current travel itinerary:
```json
{current_itinerary_str}

My follow-up request is: {follow_up_request}
\"\"\"
    react_messages.append({"role": "user", "content": user_message_content_for_react})
else:
    print("Initial itinerary not available. Skipping ReAct agent interaction for now.")
    # You could add a general user request here if you want the ReAct agent to build from scratch
    # react_messages.append({"role": "user", "content": "Help me plan a 3-day cultural trip to AgentsVille for a mid-range budget."})


if initial_itinerary or not initial_itinerary: # Allow running even if initial itinerary failed, for general queries
    MAX_TURNS = 7
    final_answer_received = False

    print("\\n--- Starting ReAct agent interaction ---")
    if len(react_messages) > 1: # if there's a user message
        print_messages([react_messages[-1]])
    else:
        print("No initial user message for ReAct agent.")


    for turn in range(MAX_TURNS):
        if len(react_messages) <= 1 and not initial_itinerary: # Safety break if no user input
            print("Stopping ReAct loop: No user query to process.")
            break
            
        print(f"\\n--- Agent Turn {turn + 1} ---")
        
        llm_response_message_obj = get_completion(react_messages) # Returns message object or content string

        if not llm_response_message_obj:
            print("Agent did not provide a response. Ending interaction.")
            break
        
        current_ai_response_content = None
        current_ai_tool_calls = None

        if hasattr(llm_response_message_obj, 'tool_calls') and llm_response_message_obj.tool_calls:
            current_ai_tool_calls = llm_response_message_obj.tool_calls
            # For OpenAI v1.x, the content might be None when tool_calls are present
            current_ai_response_content = llm_response_message_obj.content
            
            # Construct the message to append to history, including tool_calls
            assistant_message_to_append = {"role": "assistant", "content": current_ai_response_content}
            # The OpenAI library's message object for tool_calls is a list of ChatCompletionMessageToolCall objects.
            # We need to convert it to the dict format expected by the API for subsequent calls.
            assistant_message_to_append["tool_calls"] = [
                {"id": tc.id, "type": tc.type, "function": {"name": tc.function.name, "arguments": tc.function.arguments}}
                for tc in current_ai_tool_calls
            ]
            react_messages.append(assistant_message_to_append)

        elif isinstance(llm_response_message_obj, str):
            current_ai_response_content = llm_response_message_obj
            react_messages.append({"role": "assistant", "content": current_ai_response_content})
        else:
            print("Error: Unexpected response type from LLM that is not string or tool_call capable message object.")
            break # Exit loop on unexpected response type

        print_messages([react_messages[-1]]) # Display AI's full response (THINK/ACT part)

        # TODO: Implement the logic to handle the agent's response.
        # This is the most complex part for you to build, applying what you learned about ReAct.
        #
        # 1. Check if `current_ai_response_content` (if it's a string) contains "FINAL ANSWER:".
        #    If yes, set `final_answer_received = True` and `break` the loop.
        #
        # 2. If `current_ai_tool_calls` is present (meaning the LLM wants to use tools):
        #    - Iterate through each `tool_call` in `current_ai_tool_calls`.
        #    - Get `tool_name`, `tool_id`, and `tool_arguments_str` (which is a JSON string).
        #    - Parse `tool_arguments_str` into a Python dictionary.
        #    - Look up `tool_name` in your `available_tools` dictionary to get the actual Python function.
        #    - Call the Python tool function with the parsed arguments: `tool_function(**parsed_arguments)`.
        #    - Take the result from the Python tool function, convert it to a JSON string.
        #    - Construct a 'tool' role message to append to `react_messages`:
        #      `{"role": "tool", "tool_call_id": tool_id, "name": tool_name, "content": tool_result_json_string}`
        #    - Print this tool message using `print_messages`.
        #
        # 3. If it's not a final answer and not a tool call (i.e., `current_ai_response_content` is text but no "FINAL ANSWER:"):
        #    This might be the LLM asking a clarifying question or just thinking aloud without a proper ACT.
        #    For this project, you can append a generic "OBSERVATION: Please proceed by calling a tool or providing a FINAL ANSWER."
        #    message from the 'user' role to `react_messages` to prompt the LLM again.
        #    Or, if the content seems like a partial thought, you might just let the loop continue.

        # --- START TODO: Implement ReAct Loop Logic (Parsing ACT, Calling Tools, Appending Observations) ---
        
        if current_ai_response_content and "FINAL ANSWER:" in current_ai_response_content:
            pass # Replace with your logic
        elif current_ai_tool_calls:
            pass # Replace with your logic
        else:
            pass # Replace with your logic

        # Example of how you might structure the tool call handling (you need to complete this):
        # if current_ai_tool_calls:
        #     for tool_call_obj in current_ai_tool_calls:
        #         tool_name = tool_call_obj.function.name
        #         tool_id = tool_call_obj.id
        #         tool_args_str = tool_call_obj.function.arguments
        #         
        #         print(f"  Attempting to call tool: {tool_name} with ID: {tool_id}")
        #         # ... (parse args_str, find function, call it, create tool message, append to react_messages) ...
        #         # Remember to handle potential errors during parsing or tool execution.
        
        # --- END TODO: Implement ReAct Loop Logic ---

    if not final_answer_received:
        print(f"\\nAgent did not provide a final answer after {MAX_TURNS} turns.")


# --- 4. Final Output and Reflection ---
print("\\n--- Full ReAct Agent Conversation History (if run) ---")
if initial_itinerary or len(react_messages) > 1 : # only print if react loop was entered
    print_messages(react_messages)

# TODO: If the agent modified the itinerary or provided specific information in its FINAL ANSWER,
# you might want to parse and display that here. For now, reviewing the printed conversation is key.

# --- Reflection Questions: ---
# (Keep the reflection questions as provided in the original starter)

# --- Congratulations! ---
# (Keep the congratulations message as provided)

  ```